# 1. Extração dos Enunciados das Provas do Enem LEDOR (2015-2023)

Este notebook apresenta o processo de extração textual das provas do ENEM adaptadas para acessibilidade, conhecidas como provas LEDOR. Essas versões são destinadas a participantes com deficiência visual e trazem descrições detalhadas de imagens e gráficos, além de uma estrutura mais textual e padronizada.

Optamos por utilizar as provas LEDOR ao invés da aplicação regular devido aos seguintes benefícios para a nossa tarefas de PLN e predição:

- Enunciados mais completos e com menos dependência de elementos visuais;
- Formato mais limpo, facilitando a tokenização, análise semântica e a modelagem textual;
- Melhor desempenho na extração automática com bibliotecas como PyMuPDF, em conjunto com regex e revisão manual.


In [8]:
# Importando as bibliotecas necessárias para extração de texto
import re
import pandas as pd
from PyPDF2 import PdfReader
import pymupdf
from PIL import Image
import io
import pytesseract
import fitz
import os
from pdf2image import convert_from_path

---
## 1.1. Funções Auxiliares

In [9]:
# Função que recebe o caminho do pdf, a página de início e a de fim e retorna as questões
def extract_questions(full_text: str, caps_lock: bool = False) -> list[str]:
    delimiter = "QUESTÃO " if caps_lock else "Questão "
    sections = full_text.split(delimiter)
    questions = [s[3:] for s in sections if s[:2].isnumeric()]
    return questions

# Extração dos enunciados
def get_questions_text(pdf_path: str, start_page: int, end_page: int, caps_lock: bool = False) -> list[str]:
    reader = PdfReader(pdf_path)
    final_questions = []

    for page_num in range(start_page, end_page):
        full_text = reader.pages[page_num].extract_text() or ""
        final_questions.extend(extract_questions(full_text, caps_lock))
    return final_questions


In [10]:
# def extract_questions_2021(full_text: str, caps_lock: bool = False) -> list[str]:
#     delimiter = "QUESTÃO " if caps_lock else "Questão "
#     sections = full_text.split(delimiter)
#     questions = [s[3:] for s in sections if s[:2].isnumeric()]
#     return questions

# # Função para processar PDF imagem (com OCR)
# def get_questions_text_2021(pdf_path: str, start_page: int, end_page: int, caps_lock: bool = False) -> list[str]:
#     pages = convert_from_path(pdf_path, dpi=300)
#     final_questions = []

#     for i in range(start_page, end_page):
#         image = pages[i]
#         full_text = pytesseract.image_to_string(image, lang="por") or ""
        
#         rodape_pattern = r"(\d+\s*)?CH\s*-?\s*1º\s*dia\s*\|\s*Caderno\s*9\s*-?\s*LARANJA\s*LEDO[R]*\s*-?\s*1ª?\s*Aplicação(\s*\d+)?"
#         full_text = re.sub(rodape_pattern, "", full_text)


#         # Remover códigos do rodapé, por exemplo: *020325AZ5*
#         full_text = re.sub(r"\*\w+\*", "", full_text)

#         # Remover linhas que contenham apenas dígitos (possíveis números de página)
#         full_text = re.sub(r"\n\s*\d+\s*$", "", full_text)        

#         final_questions.extend(extract_questions_2021(full_text, caps_lock))
#         print(full_text)


#     return final_questions

In [11]:
# Função que retorna as questões sem as alternativas e as alternativas formatadas
def format_alternatives(alt: str) -> str:
    match = re.search(r"(A\s.+?)(B\s.+?)(C\s.+?)(D\s.+?)(E\s.+)", alt)
    if match:
        groups = match.groups()
        return "; ".join([f"{item[0]}: {item[2:-1]}" for item in groups])
    return ""


def get_alternatives(questions: list[str], filter: bool = False) -> tuple[list[str], list[str]]:
    formatted_questions = []
    formatted_alternatives = []

    for question in questions:
        parts = re.split(r"(\nA\s.*\n)", question)

        alternatives_text = "".join(parts[-2:]).replace("\n", "")
        alternatives_text = re.sub(r"\*.*\*", "", alternatives_text)
        alternatives_text = re.sub(r"(CH).*\d", "", alternatives_text)
        alternatives = format_alternatives(alternatives_text)

        question_text = "".join(parts[:-2]).replace("\n", "")

        formatted_questions.append(question_text)
        formatted_alternatives.append(alternatives)

    return formatted_questions, formatted_alternatives

In [12]:
# Função que realiza a leitura dos microdados
def reading_data(path_to_data: str, year: str, code: int) -> pd.DataFrame:

    df = pd.read_csv(path_to_data, sep=';', encoding='latin-1')
    
    df = df[(df['SG_AREA'] == 'CH') & (df['TX_COR'].str.lower() == 'laranja')]
    df = df[df['CO_PROVA'] == code]
    df = df[['CO_POSICAO', 'TX_GABARITO', 'NU_PARAM_A', 'NU_PARAM_B', 'NU_PARAM_C']].copy()
    
    df['ANO'] = year

    return df

# Função auxiliar para criar o dataset com as novas colunas

def merge_questions_alternatives(dataset: pd.DataFrame, questions: list[str], alts: list[str]) -> pd.DataFrame:
    dataset = dataset.sort_values(by=['CO_POSICAO'])
    
    dataset['QUESTOES'] = questions
    dataset['ALTERNATIVAS'] = alts

    return dataset

---
## 1.2. Extração do Texto e Cruzamento com Microdados

In [13]:
years = {
    2017: (19, 32, True, 408),
    2018: (19, 32, True, 464),
    2019: (19, 32, False, 520),
    2020: (19, 32, False, 574),
#     2021: (23, 34, False, 886),
    2022: (20, 32, True, 1062),
    2023: (19, 32, True, 1198),
}

In [14]:
for year, (start_page, end_page, question_caps, code) in years.items():
    path_to_microdados = f"../data/raw/microdados/microdados_{year}.csv"
    path_to_prova = f"../data/raw/provas/Enem_{year}.pdf"
    output_path = f"../data/processed/enem_{year}.csv"

    questions = get_questions_text(path_to_prova, start_page, end_page, question_caps)
    questions, alternatives = get_alternatives(questions)

    
    dataset = reading_data(path_to_microdados, year, code)

    if year in (2022, 2023):
        questions = [question for question in questions if question]
        alternatives = [alternative for alternative in alternatives if alternative]
        
    dataset = merge_questions_alternatives(dataset, questions, alternatives)

    dataset = dataset.rename(
        columns={
            "CO_POSICAO": "numero_questao",
            "QUESTOES": "enunciado",
            "ALTERNATIVAS": "alternativas",
            "NU_PARAM_B": "nu_param_B",
            "TX_GABARITO": "gabarito",
        }
    )

    dataset.to_csv(output_path, sep=";", index=False)
    print(f"Arquivo {output_path} criado com sucesso.")

Arquivo ../data/processed/enem_2017.csv criado com sucesso.
Arquivo ../data/processed/enem_2018.csv criado com sucesso.
Arquivo ../data/processed/enem_2019.csv criado com sucesso.
Arquivo ../data/processed/enem_2020.csv criado com sucesso.
Arquivo ../data/processed/enem_2022.csv criado com sucesso.
Arquivo ../data/processed/enem_2023.csv criado com sucesso.


---
## 1.3. Exceção: Prova de 2021

In [15]:
# input_path = "../data/raw/provas/Enem_2021.pdf"

# doc = pymupdf.open(input_path)

# texto_total = ""

# for pagina in doc[23:34]:
#     # Renderiza a página como imagem
#     pix = pagina.get_pixmap(dpi=300)
#     img = Image.open(io.BytesIO(pix.tobytes()))

#     # OCR para extrair texto da imagem
#     texto = pytesseract.image_to_string(img, lang="por")
#     texto_total += texto + "\n"
#     print('teste')

# doc.close()

# # Remover o rodapé com variações
# rodape_pattern = r"(\d+\s*)?CH\s*-?\s*1º\s*dia\s*\|\s*Caderno\s*9\s*-?\s*LARANJA\s*LEDO[R]*\s*-?\s*1ª?\s*Aplicação(\s*\d+)?"
# full_text = re.sub(rodape_pattern, "", full_text)


# # Remover códigos do rodapé, por exemplo: *020325AZ5*
# full_text = re.sub(r"\*\w+\*", "", full_text)

# # Remover linhas que contenham apenas dígitos (possíveis números de página)
# full_text = re.sub(r"\n\s*\d+\s*$", "", full_text)

# # Dividindo o texto em blocos de questões

# question_blocks = re.split(r"QUESTÃO\s+", full_text)
# question_blocks = [block.strip() for block in question_blocks if block.strip()]

# results = []

# print(full_text)

# for block in question_blocks:
#     # Extraindo número da questão
#     m = re.match(r"(\d+)", block)
#     if not m:
#         continue
#     num_quest = m.group(1)

#     content = block[m.end() :].strip()

#     content = content.replace("\x03", "").replace("\x0f", "").replace("\x11", "")

#     # Separando enunciado e alternativas.
#     padrao = r"(A[^\n]*\n.*?\nB[^\n]*\n.*?\nC[^\n]*\n.*?\nD[^\n]*\n.*?\nE[^\n]*\n.*?)(?=\n\n|\Z)"
#     match = re.search(padrao, content, flags=re.DOTALL)
#     if match:
#         enunciado = content[: match.start()].strip()
#         alt_text = match.group(0).strip()
#     else:
#         enunciado = content
#         alt_text = ""

#     # Extraindo alternativas
#     alt_regex = r"\b([A-E])\b[\s\-\.:]*([\s\S]*?)(?=\n[A-E]\b[\s\-\.:]|$)"
#     alt_raw = re.findall(alt_regex, alt_text, flags=re.DOTALL)

#     # Formatando alternativas como "A: texto"
#     alternativas = "; ".join(
#         [f"{letter}: {text.strip()}" for letter, text in alt_raw]
#     )
#     results.append(
#         {
#             "numero_questao": num_quest,
#             "enunciado": enunciado,
#             "alternativas": alternativas,
#         }
#     )

# # with open(output_path, "w", newline="", encoding="utf-8") as csvfile:
# #     fieldnames = ["numero_questao", "enunciado", "alternativas"]
# #     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
# #     writer.writeheader()
# #     for row in results:
# #         writer.writerow(row)

# print(results)

---
## 1.4. Unindo Todos os Dados em Único Arquivo

In [16]:
dataframes = {}
output_path = "../data/final/enem_data.csv"

for year in years:
    df_path = f"../data/processed/enem_{year}.csv"
    df = pd.read_csv(df_path, encoding="utf-8", quotechar='"', sep=';', index_col=0)
    dataframes[year] = df

In [17]:
df_final = pd.concat(dataframes.values(), axis=0)

df_final.to_csv(output_path)